In [ ]:
!docker build -t 715867985850.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310 -f Dockerfile.train .

In [ ]:
!docker images

In [ ]:
!docker push 715867985850.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310

In [ ]:
from sagemaker import get_execution_role
from sagemaker.estimator import Estimator
import os

role=get_execution_role()

hyperparameters={'epochs': 1}

estimator=Estimator(
    image_uri='715867985850.dkr.ecr.us-east-1.amazonaws.com/tensorflow-training:2.12.0-cpu-py310',
    role=role,
    instance_count=1,
    instance_type='ml.p2.xlarge',
    hyperparameters=hyperparameters,
    output_path='s3://itfilippoconsole/data/output'
)

print('##### ESTIMATOR FIT STARTED')
estimator.fit('s3://itfilippoconsole/data/input/house_price_regression_dataset.csv')
print('##### ESTIMATOR FIT COMPLETED')

In [ ]:
from sagemaker import get_execution_role
import boto3
import datetime
from time import gmtime, strftime

# https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints-deployment.html

#Set up autenticazione
my_session = boto3.session.Session()
aws_region = my_session.region_name
sagemaker_client = boto3.client('sagemaker', region_name=aws_region)
sagemaker_role = get_execution_role()

model_name = 'Il-top'

# Create model
create_model_response = sagemaker_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = sagemaker_role,
    PrimaryContainer = {
        'Image': '715867985850.dkr.ecr.us-east-1.amazonaws.com/tensorflow-inference:mondo',
        'ModelDataUrl': 's3://trainjob/data/output/training_job-2024-11-05-14-05-20/output/model(3).tar.gz',
    })



In [ ]:
# Create an endpoint config name. Here we create one based on the date  
# so it we can search endpoints based on creation time.
endpoint_config_name = 'la-mia-prima-api-inferenza-config'

instance_type = 'ml.p2.xlarge'

endpoint_config_response = sagemaker_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    # You will specify this name in a CreateEndpoint request.
    # List of ProductionVariant objects, one for each model that you want to host at this endpoint.
    ProductionVariants=[
        {
            "VariantName": "variant1", # The name of the production variant.
            "ModelName": model_name,
            "InstanceType": instance_type, # Specify the compute instance type.
            "InitialInstanceCount": 1 # Number of instances to launch initially.
        }
    ]
)

print(f"Created EndpointConfig: {endpoint_config_response['EndpointConfigArn']}")

# The name of the endpoint. The name must be unique within an AWS Region in your AWS account.
endpoint_name = 'la-mia-prima-api-inferenza'

create_endpoint_response = sagemaker_client.create_endpoint(
                                            EndpointName=endpoint_name, 
                                            EndpointConfigName=endpoint_config_name)